<a href="https://colab.research.google.com/github/SeoyoungOhMe/skt-fly-ai-challenger-class/blob/master/LC_06_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai langchain pypdf tiktoken sentence_transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
import os

os.environ['OPENAI_API_KEY'] = '8c91c5c7058847f3820f9d7c12bbb1fb'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://sktfly-ai.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [14]:
# Data의 loading
import openai
import tiktoken

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf')
loader.load_and_split()

[Document(page_content='2021.05\n이노비즈 정책브리프\nI n n o b i z  P o l i c y  B r i e f\n탄소중립 대응을 위한\n정부 정책과 동향\n탄소중립 대응을 위한 정부 정책과 동향\n–\n2021-04 호\n–\n발행처\n이노비즈정책연구원\n–\n발행인\n임병훈\n–\n편집인\n김세종\n–\nhttp://www.innobiz.or.kr', metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf', 'page': 0}),
 Document(page_content='02\n※ ‘2050 탄소중립 추진전략 ’을 중심으로 작성된 이노비즈 정책브리프 2021-04 호 이후 탄소중립 대응에 필요한 정책 기조와 글로벌 \n동향, 조사연구 결과 등을 꾸준히 소개할 예정입니다 .목 차\n탄소중립과 대응 필요성 1\n탄소중립 대응을 위한 정부 정책 3\n해외 주요국 동향 6\n국내 대기업 동향 7\n중소기업의 탄소중립 대응 8\n부록: 주요 정부 정책과제 추진 일정 10\n부록: 탄소중립 관련 정보 제공 누리집 12', metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf', 'page': 1}),
 Document(page_content="요 약\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있\n는 가운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨\n- '교토의정서 ’와 ‘파리기후변화협정 ’ 채택을 통해 세계 대부분 국가들이 탄소중립을 위한 목\n표와 전략을 수립하여 실천 중에 있음\n∙우리나라 정부도 지난 2020년 12월 7일 관계부처 합동으로 ‘2050 탄소중립 추진전략 ’을 발\n표하고 탄소중립을 실현하겠다는 의지를 표명함\n- 정부는 탄소중립을 실현하는 가운데 지속가능한 경제성장과 삶의 질 향

In [7]:
pages = loader.load_and_split()

In [8]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
                  chunk_size=500,
                  chunk_overlap=50,
                  length_function=tiktoken_len
)

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name='jhgan/ko-sbert-nli'
model_kwargs={'device':'cpu'}
encode_kwargs={'normalize_embeddings':True}

hf = HuggingFaceEmbeddings(
        model_name = model_name,
        model_kwargs = model_kwargs,
        encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# split
docs = text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, hf)

In [15]:
# 언어 모델의 생성
chatgpt = AzureChatOpenAI(
    deployment_name = 'dev-gpt-35-turbo'
)

In [16]:
qa = RetrievalQA.from_chain_type(
    llm = chatgpt,
    chain_type = 'stuff',
    retriever = db.as_retriever(
                                    search_type = 'mmr',
                                    search_kwargs = {'k':3,'fetch_k':10}
    ),
    return_source_documents=True
)

In [17]:
query = '미국의 탄소중립은?'

In [18]:
result = qa(query)
result

{'query': '미국의 탄소중립은?',
 'result': '미국은 2030년까지 2005년 대비 50% 감축하는 목표를 설정했습니다. 또한, 기후정상회의에 참석한 문재인 대통령은 온실가스 감축 목표를 추가 상향해 올해 안에 UN에 제출할 것이라고 밝혔습니다.',
 'source_documents': [Document(page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨', metadata={'page': 3, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf'}),
  Document(page_content='터 수입한 상품에 비용을 부과하는 식으로 자국 산업을 보호하겠다고 선언한 상태임\n∙무역 의존도가 높고 제조업 비중, 탄소 다배출 업종(철강, 석유화학 등) 비중이 높은 우리나라의 \n산업 구조를 생각해 본다면 탄소중립 대응이 결코 쉬운 과제는 아닐 것임\n- 기업의 경우 고탄소에서 저탄소로 산업구조가 변경됨에 따라 부담

In [19]:
query = '탄소중립에 중소기업이 필요한 이유는?'

In [20]:
result = qa(query)
result

{'query': '탄소중립에 중소기업이 필요한 이유는?',
 'result': '탄소중립에 중소기업의 참여가 필요한 이유는 국가 경제에서 중소기업이 차지하는 비중이 높아서 정부와 몇몇 대기업의 노력만으로는 탄소중립을 실현하기 어렵기 때문입니다. 따라서 중소기업의 적극적 참여가 필요하며, 이를 유인할 수 있는 정부 정책과 제도가 필요합니다. 중소기업이 탄소중립에 적극적으로 참여하면 경쟁력을 확보하고 성장할 수 있는 동력을 마련할 수 있을 것으로 기대됩니다.',
 'source_documents': [Document(page_content='대한 대응 필요성을 느끼지 못한다는 결과를 보여주고 있음\n∙향후 중소기업을 대상으로 도입될 정부 정책과 제도에 대한 관심과 적극적 참여가 필요함 \n- 국가 경제에서 중소기업이 차지하는 비중이 높은 우리나라는 정부와 몇몇 대기업의 노력만으로 탄소중립\n을 실현하기 불가능함\n- 따라서 탄소중립에 대한 중소기업의 적극적 참여가 필요하며 , 중소기업의 참여를 유인할 수 있는 다양한 \n정부 정책과 제도가 마련될 예정임\n- 지원책을 적극 활용하여 많은 중소기업이 친환경 기술 및 제품 개발, 사업 전환 등에 적극적으로 참여한\n다면, 탄소중립 시대에서 경쟁할 수 있는 역량을 확보하고 성장할 수 있는 동력을 마련할 수 있을 것임', metadata={'page': 11, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf'}),
  Document(page_content='속화 추진\n- 산업 분야에서는 재생에너지 , 수소, 에너지 IT 등 3대 에너지 신산업 육성에 집중\n- (고탄소 산업구조 혁신) 철강, 석유화학 등 탄소 다배출 업종에 대한 기술개발 지원, 고탄소 중소기업 대\n상 맞춤형 공정개선 지원 등을 통해 저탄소 산업구조로의 전환 추진\n- 금속･화학제품제조업 등 다배출 중소기업을 대상으로 공정진단을 통해 맞춤형 공정개선 ‧설비보급을 지원\n하는 등 산